## Package Imports

In [10]:
#multiprocess inference: 
    #there is one analyze process running throughohut the duration of the program
        #main adds the microphone_data to a queue which analyze gets from the queue and analyzes
    #there is also one sms process running throughout the duration of the program
        #if a gunshot is detected w prob > 90%, a 1 is added to a queue which the sms alert process will
        #take as a sign to send a text

import pyaudio
import librosa
import logging
import multiprocessing
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from multiprocessing import Process, active_children
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import IPython.display as ipd
import matplotlib.pyplot as plt
#from gsmmodem.modem import GsmModem
import time
import soundfile as sf
from array import array
from sys import byteorder


## Set up Logging

In [2]:
#stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger('debugger')
logger.setLevel(logging.DEBUG)
ch = logging.FileHandler('spam.log')
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

## Variable Initializations

In [3]:
#on the pi, change to paInt16
audio_format = pyaudio.paInt16
audio_rate = 44100
audio_channels = 1
audio_device_index = 0
audio_frames_per_buffer = 22050
audio_sample_duration = 2
input_shape = (audio_rate, 1)
phone_numbers_to_message = ["8163449956", "9176202840", "7857642331"]

## Normalization Function

In [4]:
#normalization function
def normalize(sound_data):
    # Averages the volume out
    sound_normalization_threshold = 16384
    times = float(sound_normalization_threshold) / max(abs(i) for i in sound_data)
    
    r = array('h')
    for datum in sound_data:
        r.append(int(datum * times))
    return np.array(r)

## Process 1: Analuze

In [16]:
#loads model
#if microphone data reaches a threshold, predicts with the model
#if model predicts a gunshot, send a text, and add data to the time localization queue

def analyze_microphone_data(audio_rate, ):

    #LOAD THE MODEL
    def auc(y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred)[1]
        K.get_session().run(tf.local_variables_initializer())
        return auc

    drop_out_rate = 0.1
    learning_rate = 0.001
    number_of_epochs = 100
    number_of_classes = 2
    batch_size = 32
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    input_tensor = Input(shape=input_shape)
    metrics = [auc, "accuracy"]
    
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(16)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(1028, activation="relu")(x)
    output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

    model = tf.keras.Model(input_tensor, output_tensor)
    model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)
    
    model.load_weights("./models/gunshot_sound_model.h5")

    
    #infinite loop
    while True:
        #will wait until something is in the queue to continue
        microphone_data = audio_analysis_queue.get()
        time_of_sample = audio_analysis_queue.get()
        
        # Performs post-processing on live audio samples
        reformed_microphone_data = librosa.resample(y=microphone_data, orig_sr=audio_rate, target_sr=22050)
        reformed_microphone_data = normalize(reformed_microphone_data)
        reformed_microphone_data = reformed_microphone_data[:audio_rate]
        reformed_microphone_data = reformed_microphone_data.reshape(-1, audio_rate, 1)

        # Passes a given audio sample into the model for prediction
        probabilities = model.predict(reformed_microphone_data)
        emit_string = "Probabilities derived by the model: " + str(probabilities)
        logger.debug(emit_string)

        #send a text if gunshot if detected
        if (probabilities[0][1] >= 0.9):
            sms_alert_queue.put("1")
            time_localization_queue.put(reformed_microphone_data.reshape(44100))
            time_localization_queue.put(time_of_sample)


## Process 3: Figure out exact time of gunshot

In [17]:
#get data from the time localization queue
#use a threshold in the data to figure out where the spikes (gunshots) are at in the clip

def localize_time():
    
    #infinite loop
    while True:
        #get data and time at beginning of data chunk
        mic_data = time_localization_queue.get()
        time_at_beg = time_localization_queue.get()

        #sort the data, figure out the threshold
        sorted_data = np.sort(mic_data)
        threshold = sorted_data[len(sorted_data) - int(len(sorted_data)*0.001)]

        #find all values above that threshold
        above_threshold = []
        for i in range(0, len(mic_data)):
            if mic_data[i] > threshold:
                above_threshold.append(i)
                
        #separate out individual gunshots from that whole chunk
        distinct_shots = []
        distinct_shots.append(above_threshold[0])
        for i in range(1, len(above_threshold)):
            
            if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
                distinct_shots.append(above_threshold[i])

        logger.debug("There were " + str(len(distinct_shots)) + " distinct shots detected at " + str(time.ctime(time_at_beg)))
        for i in distinct_shots:
            logger.debug(i/22050)
            
        logger.debug("These ")

        #write out a soundfile of the entire chunk
        with sf.SoundFile("/Users/laurenogden/Downloads/" + str(time.ctime(time_at_beg)) + ".wav", mode='wb', samplerate=22050, channels=1) as file:
            file.write(mic_data)


## Process 2: Send SMS

In [18]:
# send an sms if a gunshot is detected

def send_sms_alert(phone_numbers_to_message, ):
    
    # Modem variables
    modem_port = '/dev/ttyUSB0'
    modem_baudrate = 115200
    modem_sim_pin = None # SIM card PIN (if any)
    
    # Connect to the modem
    #print("Initializing connection to modem...")
    #modem = GsmModem(modem_port, modem_baudrate)
    #modem.smsTextMode = False
    #modem.connect(modem_sim_pin)
    
    
    # If the model detects a gunshot, an SMS alert will be sent to local authorities
    while True:
        sms_alert_status = sms_alert_queue.get()
        if sms_alert_status == "1":
            logger.debug("~~~~~~~~pretend like I just sent a text message~~~~~~~~~~")
            '''
            try:
                modem.waitForNetworkCoverage(timeout=86400)
                message = " (Testing) ALERT: A Gunshot Has Been Detected (Testing)"
                for number in phone_numbers_to_message:
                    modem.sendSms(number, message)
                logger.debug(" *** Sent out an SMS alert to all designated recipients *** ")
            except:
                logger.debug("ERROR: Unable to successfully send an SMS alert to the designated recipients.")
                pass
            finally:
                logger.debug(" * Finished evaluating an audio sample with the model * ")
            '''

## Main Process: Capture Audio

In [19]:
#open the stream
pa = pyaudio.PyAudio()
    
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True)

In [20]:
if __name__ == '__main__':
    
    #Create the Queues
    sms_alert_queue = multiprocessing.Queue()
    audio_analysis_queue = multiprocessing.Queue()
    time_localization_queue = multiprocessing.Queue()
    
    #Create Analysis Process
    analysis_process = Process(target = analyze_microphone_data, args = (audio_rate, ))
    analysis_process.start()
    
    #Create SMS Process
    sms_alert_process = Process(target = send_sms_alert, args = (phone_numbers_to_message, ))
    sms_alert_process.start()
    
    #Create process for localization
    time_localization_process = Process(target = localize_time, args = ())
    time_localization_process.start()
    

    logger.debug("----------------------------------- RECORDING AUDIO -------------------------------------------")
    
    while(True):
        np_array_data = []
        
        now = time.time()

        # Loops through the stream and appends audio chunks to the frame array
        for i in range(0, int(audio_rate / audio_frames_per_buffer * audio_sample_duration)):
            data = stream.read(audio_frames_per_buffer, exception_on_overflow = False)
            #if on the pi, change dtype to int16
            np_array_data.append(np.frombuffer(data, dtype=np.int16))
        microphone_data = np.concatenate(np_array_data)
        #emit_string = "Cumulative length of a given two-second audio sample: " + str(len(microphone_data))
        #logger.debug(emit_string)
        #emit_string = "The maximum frequency value for a given two-second audio sample: " + str(max(microphone_data))
        #logger.debug(emit_string)
        
        
        # put microphone data on the queue to go to the analysis process
            # the analysis process will check the threshold: if max(microphone_data) >= 1000:
        audio_analysis_queue.put(microphone_data)
        #also put time: for localization
        audio_analysis_queue.put(now)

        
        # Closes all finished processes    
        kids = active_children()
        #logger.debug("these are my active children: ")
        #logger.debug(kids)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
3324
13369
6518
7871
7734
11456
13564
10270
12555


Process Process-9:
Process Process-7:
Process Process-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/proc

KeyboardInterrupt: 